为了代码更通用且具有可复用价值构建了一个向量数据库 Chroma
，存储过程中的向量

In [ ]:
# !pip install -qU "langchain-chroma>=0.1.2"

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
TOP_K = 10

In [3]:
raw_file = "data/init_values.xls"
ref_file = "data/alter_values.xls"

In [4]:
raw_df = pd.read_excel(raw_file)
ref_df = pd.read_excel(ref_file)

In [5]:
raw_texts = raw_df["行业分类"].to_list()
ref_texts = ref_df["类别名称2017"].to_list()

针对 ref_texts 里面的文本构建向量数据库

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from modelscope import snapshot_download

model_dir = snapshot_download("jinaai/jina-embeddings-v2-base-zh")

hf_embedding = HuggingFaceEmbeddings(
    model_name=model_dir, model_kwargs={"trust_remote_code": True}
)

C:\Users\1\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\1\AppData\Local\Temp\ipykernel_26640\4162136049.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedding = HuggingFaceEmbeddings(


In [7]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="industry_name",
    embedding_function=hf_embedding,
    persist_directory="./chroma_industry_name_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

documents = [
    Document(
        page_content=text,
        id=idx,
    )
    for idx, text in enumerate(ref_texts)
]

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

In [15]:
data = []

for text1 in tqdm(raw_texts):
    results = vector_store.similarity_search_with_score(text1, k=TOP_K)
    ans = []
    for res, score in results:
        ans.append(res.page_content)
    data.append(ans)

tmp_df = pd.DataFrame(data, columns=list(range(TOP_K)))
ans_df = pd.concat([raw_df, tmp_df], axis=1)
ans_df.to_csv("向量筛选.csv", index=False, encoding="utf-8")

100%|██████████| 143/143 [00:02<00:00, 50.88it/s]
